In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
berk.csv
berk-test.csv
GoogleLenetv2.png
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
samples
state_transition_matrix.npy
test_predictions.csv
title.csv


# Preparing attributes_list & labels

In [3]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_RAP/RAP_annotation/RAP_annotation.mat')['RAP_annotation']
attributes_list = []
for i in range(data['attribute_eng'][0][0].shape[0]):
    attributes_list.append(data['attribute_eng'][0][0][i][0][0])
print(attributes_list)
attributes_list_chinese = []
for i in range(data['attribute_chinese'][0][0].shape[0]):
    attributes_list_chinese.append(data['attribute_chinese'][0][0][i][0][0])
print(attributes_list_chinese)

['Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'hs-BaldHead', 'hs-LongHair', 'hs-BlackHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand', 'faceFront', 'faceBack', 'faceLeft', 'faceRight', 'occlusionLeft', 'occlusionRight', 'occlusionUp', 'occlusionDown', 'occlusion-Environment', 'occlusion-Attachment', 'occlusion-Person', 'occlusion-Other', 'up-Black

In [4]:
low_level = [11]
mid_level = [9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
high_level = [0,1,2,3,4,5,6,7,8,43,44,45,46,47,48,49,50]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)

The total attributes labeled:  51
['hs-BlackHair', 'hs-BaldHead', 'hs-LongHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand']


In [5]:
filename = r"../results/RAP_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 51))
for i in range(length):
    data_y[i] = np.array(data[i, 1:52], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
split = np.load('../results/RAP_partion.npy').item()
y_test = data_y[list(split['test'][0])]
y_test_hiar = data_y_hiar[list(split['test'][0])]
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the y_test is:  (8317, 51)
The shape of the y_test is:  (8317, 51)


# Positive Samples

In [6]:
for i in range(data_y_hiar.shape[1]):
    print(sum(data_y_hiar[:, i]))

39288.0
158.0
7873.0
695.0
3034.0
435.0
9198.0
5133.0
2403.0
9517.0
7185.0
13107.0
921.0
1260.0
742.0
23302.0
2708.0
1164.0
1523.0
11159.0
5544.0
12578.0
11265.0
5159.0
517.0
6543.0
754.0
2669.0
1169.0
1753.0
1146.0
526.0
1019.0
12251.0
12951.0
415.0
17882.0
23288.0
6027.0
30876.0
4064.0
39352.0
2008.0
1418.0
1364.0
3818.0
1021.0
432.0
732.0
975.0
5361.0


In [7]:
print(attributes_list[11])
print(attributes_list[24])
print(attributes_list[2])

hs-BlackHair
lb-LongTrousers
Age17-30


# Ratio>0.01

In [25]:
selected_ = []
selected = []

total = len(data_y)
print(total)
for i in range(len(attributes_list_hiar)):
    if i!=1:
        selected_.append(i)
    else:
        print(attributes_list[i])
for i in range(len(attributes_list_hiar)):
    if i!=len(low_level)+len(mid_level)+1:
        selected.append(i)
    else:
        print(attributes_list_hiar[i])
print(len(selected_))
print(len(selected))

41585
AgeLess16
AgeLess16
50
50


# Defining the function

In [51]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0.).count(True)
        #print(TP, P, TN, N)
        #print(P,',', N,',', TP,',', TN)
        #if P != 0:
        res += TP/P + TN/N
        """else:
            res += 1 + TN/N"""
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            #print(y_pred[i])
            #print(y_true[i])
            #print(sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i])))
            M_ += 1
    #print(M_)
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

# Metrics

In [64]:
###hiarBayesGoogLeNet
predictions_prob = np.load(
    "../results/predictions/hiarBayesGoogLeNetv2_binary51_75v2_binary51_75v2_epoch50_valloss16.85_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

"""
###hiarGoogLeNet
predictions_prob = np.load(
    "../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
label = y_test_hiar[:, selected]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))
"""

###GoogLeNet
predictions_prob = np.load(
    "../results/predictions/GoogLeNetv2_binary51_75v2_binary51_b4_75v2__epoch50_valloss11.58.h_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.6149738280352659
0.583368989437
0.8018574110884947
0.663640742682157
0.726231208754208
GoogLeNet
0.6365683300056856
0.5889426033149193
0.8077392561882055
0.6675404632239964
0.730978173352664


In [50]:
mA(predictions, label)

805.0 2626.0 4034 5691.0
1.0 87.0 8172 8230.0
1197.0 3632.0 3135 4685.0
2859.0 4598.0 1420 3719.0
65.0 1192.0 6692 7125.0
5718.0 6176.0 146 2141.0
17.0 843.0 7345 7474.0
7530.0 7867.0 24 450.0
21.0 413.0 7564 7904.0
0.0 36.0 8276 8281.0
322.0 1599.0 5363 6718.0
7680.0 7845.0 11 472.0
4.0 166.0 8035 8151.0
7.0 578.0 7632 7739.0
0.0 70.0 8242 8247.0
395.0 1807.0 5066 6510.0
123.0 1008.0 6644 7309.0
14.0 518.0 7586 7799.0
470.0 1912.0 4795 6405.0
139.0 1421.0 6238 6896.0
441.0 2617.0 4686 5700.0
6.0 201.0 7959 8116.0
5.0 255.0 7909 8062.0
3.0 148.0 8033 8169.0
2478.0 4665.0 1702 3652.0
18.0 554.0 7326 7763.0
1.0 221.0 7912 8096.0
6.0 329.0 7841 7988.0
518.0 2228.0 4684 6089.0
115.0 1114.0 6413 7203.0
669.0 2486.0 4268 5831.0
367.0 2269.0 5030 6048.0
128.0 1045.0 6444 7272.0
0.0 102.0 8215 8215.0
2.0 1272.0 7042 7045.0
0.0 147.0 8135 8170.0
11.0 505.0 7726 7812.0
3.0 231.0 7855 8086.0
3.0 337.0 7872 7980.0
0.0 247.0 8051 8070.0
0.0 114.0 8197 8203.0
4.0 199.0 7926 8118.0
603.0 2454.0 4411 

0.4997596787039186

# Attributes

In [55]:
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary51_trainable_binary51_75_epoch50_valloss0.22_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

"""
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})
"""

###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_b4_75_trainable_noweightloss__epoch75_valloss0.22.h_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,GoogLeNet
attributes,,
hs-BlackHair,0.954070,0.924732
hs-BaldHead,0.996273,0.995070
hs-LongHair,0.917278,0.683540
hs-Hat,0.981844,0.966574
hs-Glasses,0.931105,0.918480
hs-Muffler,0.991463,0.990982
ub-Shirt,0.879043,0.656607
ub-Sweater,0.875917,0.813635
ub-Vest,0.968979,0.913791


In [59]:
print(np.mean(data_dict['GoogLeNet']))
print(np.mean(data_dict['hiarBayesGoogLeNet']))

0.8420810671268627
0.9187702013593704


In [14]:
df.to_csv("../results/metrics/binary51_imagenet_acc7600_RAP.csv")

In [42]:
def mA_attribute(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true)
    N = M - P
    TP = sum(y_pred*y_true)
    TN = list(y_pred+y_true == 0).count(True)
    #if P != 0:
    res += TP/P + TN/N
    #else:
        #res += TN/N
    return res / 2
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary51_trainable_binary51_75_epoch50_valloss0.22_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

"""
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})
"""

###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_b4_75_trainable_noweightloss__epoch75_valloss0.22.h_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,GoogLeNet
attributes,,
hs-BlackHair,0.666026,0.501136
hs-BaldHead,0.624758,0.499698
hs-LongHair,0.826324,0.499839
hs-Hat,0.559934,0.504933
hs-Glasses,0.547549,0.499142
hs-Muffler,0.499939,0.499697
ub-Shirt,0.770213,0.498391
ub-Sweater,0.603126,0.515520
ub-Vest,0.797825,0.499858


In [16]:
df.to_csv("../results/metrics/binary51_imagenet_ma7600_RAP.csv")

# In HiarBayesGoogLeNet, Select one each level

In [21]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary3_binary3_epoch50_valloss0.40.h_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test[:, [11,24,3]]

print(predictions)
print(label)

for i in range(label.shape[1]):
    print(accuracy_score(predictions[:, i], label[:, i]))
    
for i in range(label.shape[1]):
    print(mA_attribute(predictions[:, i], label[:, i]))

[[1. 1. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 ...
 [1. 1. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 1.]
 ...
 [0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
0.9478177227365636
0.8421305759288205
0.3824696404953709
0.6308526698426072
0.8337924336926777
0.39862568115648445


In [22]:
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

0.6210902615639231
0.6732812186811066
0.8233486017357763
0.8115317720341617
0.817397481171928


# In GoogLeNet, Select one each level

In [29]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_b4_mysplit_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected_]
label = y_test[:, selected_]

print(predictions)
print(label)

for i in range(label.shape[1]):
    print(accuracy_score(predictions[:, i], label[:, i]))
    
for i in range(label.shape[1]):
    print(mA_attribute(predictions[:, i], label[:, i]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]]
0.6878682217145605
0.5467115546471083
0.4471564265961284
0.856679091018396
0.5901166285920404
0.8986413370205603
0.945653480822412
0.9503426716364074
0.9956715161716966
0.8077431766261873
0.9183599855717206
0.9800408801250451
0.9305037874233497
0.9915835036671877
0.7827341589515451
0.8788024528075027
0.9377179271371889
0.7699891787904293
0.8291451244439101
0.35241072502104126
0.9758326319586389
0.9693399062161837
0.9822051220391969
0.5016231814356138
0.9333894433088854
0.9734279187206926
0.9604424672357821
0.7321149452927739
0.8660574726463869
0.7010941445232656
0.7271852831549838
0.8743537333173019
0.9877359624864734
0.8470602380666106
0.9823253577010942
0.939280990741854
0.9722

In [30]:
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

0.49995229334997476
0.32598183494673033
0.7176405755280579
0.36504557924541114
0.48392882541009785
